<a href="https://colab.research.google.com/github/junaina/EmpathAI/blob/wip/EmpathAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded= files.upload()


Saving combined_emotion.csv to combined_emotion.csv


In [9]:
#loading dataset
import pandas as pd
df= pd.read_csv('combined_emotion.csv', sep=',', names=["text", "emotion"])
df.head()

,text,emotion
0,sentence,emotion
1,i just feel really helpless and heavy hearted,fear
2,ive enjoyed being able to slouch about relax a...,sad
3,i gave up my internship with the dmrg and am f...,fear
4,i dont know i feel so lost,sad


In [8]:
  # drop the bogus header row
df= df[df["text"]!= "sentence"]
df.reset_index(drop=True,inplace=True)
df.head()

,text,emotion
0,i just feel really helpless and heavy hearted,fear
1,ive enjoyed being able to slouch about relax a...,sad
2,i gave up my internship with the dmrg and am f...,fear
3,i dont know i feel so lost,sad
4,i am a kindergarten teacher and i am thoroughl...,fear


In [11]:
!git init
!git add .
!git commit -m "loaded dataset"
!git push -u origin wip

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@69c360db0888.(none)')
error: src refspec wip does not match any
error: failed to push some refs to 'origin'
